### Processing pipeline for 'meta_amazon_instant_video.json' 

<font color='royalblue' size=3>
<b>

Parameters for the notebook

In [12]:

datapath = '../DATA/'
df_name = "meta_Movies_and_TV"


###
###  Change here which fields will be extracted from the amazon product
###
def set_df_cell_from_product(amazonProduct, rowNumber, fieldName1, fieldName2) :
    if (amazonProduct is None) : 
        amazon_products_df.set_value(rowNumber, fieldName1, [])
        amazon_products_df.set_value(rowNumber, fieldName2, [])     
    else : 
        amazon_products_df.set_value(rowNumber, "actors",
                                     amazonProduct.actors)
        amazon_products_df.set_value(rowNumber, "directors",
                                     amazonProduct.directors)
        amazon_products_df.set_value(rowNumber, "creators", 
                                     amazonProduct.creators)
        amazon_products_df.set_value(rowNumber, "authors", 
                                     amazonProduct.authors)
        amazon_products_df.set_value(rowNumber, "model", 
                                     amazonProduct.model)
        #amazon_products_df.set_value(rowNumber, "artists",    # There is definitely no "artists" field
        #                             amazonProduct.artists)

# How frequently should we save our data to file
data_save_freq = 10000
# How many network errors we accept before we give up
network_errors_limit = 7
# feature we're interested in
features = ['asin', 'categories', 'description', 'title', 'salesRank']
columns_to_add = ['actors', 'directors', 'creators', 'authors', "model"]
json_name = df_name+'.json.gz'

In [13]:
#essential imports
import pandas as pd
import numpy as np
import json

# Sleep
import time

# Strict JSON conversion
import json 
import gzip 

# Progress display
from IPython.display import clear_output

# Amazon API querying
from amazon.api import AmazonAPI
from amazon.api import AsinNotFound

from urllib.request import HTTPError
from socket import gaierror
from urllib.request import URLError


<font color='royalblue' size=3>
<b>

Open metadata file

In [14]:
## Load all ASINs we're going to query - use the metadata files
## for this, as they contains each ASIN once and only once.
##

''' This function was provided on the amazon dataset's webpage
    It loads a gzipped file directly into a dataframe
'''
def gz_to_dataframe(datapath, filename):
    def parse(path): 
        g = gzip.open(path, 'rb') 
        for l in g: 
            yield eval(l) 
    def getDF(path): 
        i = 0 
        df = {} 
        for d in parse(path): 
            df[i] = d 
            i += 1 
        return pd.DataFrame.from_dict(df, orient='index') 
    return getDF(datapath+filename)
    
amazon_products_df = gz_to_dataframe(datapath, json_name)
amazon_products_df.head(3)

,asin,categories,description,title,price,salesRank,imUrl,related,brand
0,0000143561,"[[Movies & TV, Movies]]","3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...",12.99,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
1,0000589012,"[[Movies & TV, Movies]]",NaN,Why Don't They Just Quit? DVD Roundtable Discu...,15.95,{'Movies & TV': 1084845},http://ecx.images-amazon.com/images/I/519%2B1k...,"{'also_bought': ['B000Z3N1HQ', '0578045427', '...",NaN
2,0000695009,"[[Movies & TV, Movies]]",NaN,Understanding Seizures and Epilepsy DVD,NaN,{'Movies & TV': 1022732},http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,NaN


In [15]:
amazon_products_df = amazon_products_df[features]
amazon_products_df.head(3)

,asin,categories,description,title,salesRank
0,0000143561,"[[Movies & TV, Movies]]","3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...",{'Movies & TV': 376041}
1,0000589012,"[[Movies & TV, Movies]]",NaN,Why Don't They Just Quit? DVD Roundtable Discu...,{'Movies & TV': 1084845}
2,0000695009,"[[Movies & TV, Movies]]",NaN,Understanding Seizures and Epilepsy DVD,{'Movies & TV': 1022732}


<font color='royalblue' size=3>
<b>

Prepare for amazon api usage

In [16]:
## Sign in with amazon API 
##

def get_amazon_interface():
    f = open("api_creds")
    ar = f.read().split("\n")
    return AmazonAPI(ar[0], ar[1], ar[2])
    return ar[0], ar[1], ar[2]

amazon = get_amazon_interface()

In [17]:
## Here, we define some API query helpers
##

''' Product lookup with API, asin can be a string ('one by one' lookup)
    or a list of strings ('bulk lookup').
    bulk lookup provides better performance
'''
def get_prod(asin) : 
    if not isinstance(asin, str): 
        acc_str = asin[0]
        for e in asin : 
            acc_str += ','+e
        print(acc_str)
        return amazon.lookup(ItemId=acc_str)
    else :
        return amazon.lookup(ItemId=asin)
    
''' Splits the interval [start-end] into bulks of size bulksize
'''    
def gen_bulk_index(start, end, bulksize=10, includeEnd=False):
    size = end - start + 1
    bulks = [list(range(start+(i*bulksize), start + (i+1)*bulksize)) for i in range(0, int(size/bulksize))]
    if includeEnd : 
        bulks.append(list(range(bulks[len(bulks)-1][bulksize-1], end+1)))
    else : 
        bulks.append(list(range(bulks[len(bulks)-1][bulksize-1], end)))
    return bulks    


In [18]:
## Also, we have some functions to save the state of our data structure - in case we need to 
## to shutdown the computer and restart the query loop at a later time (not used here)
##

def save_progress(dataframe, nb_rows_processed):
    dataframe.to_csv(datapath+df_name+"_temp.csv", index=False)
    file = open(datapath+df_name+"_progress", "w")
    file.write(str(nb_rows_processed))

    
def load_progress():
    dataframe = pd.read_csv(datapath+df_name+"_temp.csv")
    file = open(datapath+df_name+"_progress", "r")
    nb_rows_processed = file.readline()
    return dataframe, int(nb_rows_processed)


<font color='royalblue' size=3>
<b>

Bulk lookups parameters and loop

In [19]:
## Parameters & initialization for bulk item lookup
##

bulksize = 10

# Change this to restore progress from file
fresh_run = True

if fresh_run : 
    # used to restart from where we were in case of unexepected network error
    lastItemLookedUp = 0
    for col in columns_to_add : 
        amazon_products_df[col] = pd.Series(dtype=object)
else : 
    amazon_products_df, lastItemLookedUp = load_progress()
    
print("last item looked up : ", lastItemLookedUp,  "  -  time : ",time.strftime("%H:%M:%S"), "\n\n")

last item looked up :  0   -  time :  02:42:40 




In [20]:
## Querying loop
##

ref_for_progress = lastItemLookedUp
lastErrorMet=lastItemLookedUp
loop_complete = False
caught_httperrors=0
caught_gaierrors=0
errors_counter=0


while not loop_complete : 
    try : 
        
        for bulk in gen_bulk_index(ref_for_progress, amazon_products_df.shape[0], bulksize=bulksize) : 
            # update progess every 100 items
            if ((bulk[0]-(ref_for_progress)) % 100 == 0) : 
                clear_output()
                print("    ",int(100 * (bulk[0]+1) / amazon_products_df.shape[0]), "% completed (",bulk[0], " rows)", "  -  time : ",time.strftime("%H:%M:%S"))
                print("         Last Item Looked up : ", lastItemLookedUp, " / ", amazon_products_df.shape[0])
                print("         HTTPErrors caught : ", caught_httperrors)
                print("         gaierrors caught : ", caught_gaierrors)
                print("\n\n\n")

            # get asins for the bulk and fetch the matching AmazonProducts
            asins = amazon_products_df['asin'][bulk].tolist()
            noAsinFound = False
            try : 
                prods = get_prod(asins)
            except AsinNotFound : 
                noAsinFound = True
                
            # if query was successful, reset error counter
            errors_counter = 0;
            
            # Then, process each product to add necessary informations in the dataframe
            if noAsinFound : 
                # Skip this bulk and reset the flag
                print("No Asin Found for bulk : ", bulk)
            elif (type(prods) is list) and (len(prods) == bulksize) :              
                # Case : we found exactly one result per ASIN
                #        process by bulk
                    for i, prod in enumerate(prods) : 
                        set_df_cell_from_product(prod, bulk[i], "actors", "directors")
            elif (type(prods) is list) or (type(prods) is AmazonApi.AmazonProduct) :  
                # Case : we obtained a list of AmazonProducts or a single AmazonProduct
                #        fallback to 1-by-1 querying
                for n in bulk :               
                    asin = amazon_products_df['asin'][n]
                    try : 
                        prod = get_prod(asin)
                    except(AsinNotFound): 
                        prod = None
                    set_df_cell_from_product(prod, n, "actors", "directors")
                    time.sleep(0.5)

            # Save progress
            lastItemLookedUp = bulk[bulksize-1]
            
            # Save data to file according to specified frequency
            if ((bulk[0]-(ref_for_progress)) % data_save_freq == 0) : 
                save_progress(amazon_products_df, lastItemLookedUp)
            
            # limit query frequency to avoid 503 errors
            time.sleep(min(bulksize/25, 5))
            
        loop_complete = True
        

    except HTTPError :
        errors_counter += 1
        caught_httperrors += 1
        # If we didn't make any progress, something must be wrong
        if errors_counter > network_errors_limit : 
            print("HTTPError caught at original_lastItemLookedUp  -  breaking")
            break 
        # else retry
        print("\n\nhttpError\n\n")
        ref_for_progress = lastItemLookedUp
        lastErrorMet = lastItemLookedUp
        
    except(gaierror, URLError): 
        errors_counter += 1
        caught_gaierrors += 1
        if errors_counter > network_errors_limit : 
            print("gaierror/urlerror caught too many times  -  breaking")
            break 
        # else retry
        print("\n\ngaierror/urlerror\n\n")
        ref_for_progress = lastItemLookedUp
        lastErrorMet = lastItemLookedUp
        

if(loop_complete):
    clear_output()    
    # save results
    amazon_products_df.to_csv(datapath+df_name+".csv")
    print("amazon query loop completed !")
    

     0 % completed ( 119  rows)   -  time :  02:43:57
         Last Item Looked up :  118  /  208321
         HTTPErrors caught :  0
         gaierrors caught :  1




0310256240,0310256240,0310257964,0310256844,0310261791,0310261988,0310261783,0310256003,0310263662,0310257883,0310268281
0310266378,0310266378,0310269148,0310269873,0310269881,0310269156,0310269458,0310271967,0310271169,0310272971,0310269164
0310271193,0310271193,0310275288,0310276772,0310276837,0310277574,0310274281,0310278899,0310278643,0310279690,0310278090
0310280796,0310280796,0310280893,0310278910,0310279658,0310278929,0310280761,0310279623,0310280923,0310282470,0310285062
0310286875,0310286875,0310287642,0310285569,0310291208,031029116X,0310291186,0310290740,0310292964,0310320224,0310292956
0310324491,0310324491,0310324173,0310292972,0310324386,0310324181,0310322197,0310326850,0310322065,0310327660,0310328225


KeyboardInterrupt: 

In [21]:
amazon_products_df

,asin,categories,description,title,salesRank,actors,directors,creators,authors,model
0,0000143561,"[[Movies & TV, Movies]]","3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...",{'Movies & TV': 376041},[],[],[],[],None
1,0000589012,"[[Movies & TV, Movies]]",NaN,Why Don't They Just Quit? DVD Roundtable Discu...,{'Movies & TV': 1084845},[],[],[],[Joe Herzanek],None
2,0000695009,"[[Movies & TV, Movies]]",NaN,Understanding Seizures and Epilepsy DVD,{'Movies & TV': 1022732},[],[],[],[Epilepsy Foundation],None
3,000107461X,"[[Movies & TV, Movies]]",NaN,Live in Houston [VHS],{'Movies & TV': 954116},[Douglas Miller],[],"[(Joe Thomas, Producer), (Joseph Williams, Pro...",[],None
4,0000143529,"[[Movies & TV, Movies]]",Disc 1: Flour Power (Scones; Shortcakes; South...,My Fair Pastry (Good Eats Vol. 9),{'Movies & TV': 463562},[Alton Brown],[],[],[],None
5,0000143502,"[[Movies & TV, Movies]]",Rise and Swine (Good Eats Vol. 7) includes bon...,Rise and Swine (Good Eats Vol. 7),{'Movies & TV': 467172},[Alton Brown],[],[],[],None
6,0000143588,"[[Movies & TV, Movies]]",Barefoot Contessa Volume 2: On these three dis...,"Barefoot Contessa (with Ina Garten), Entertain...",{'Movies & TV': 309405},[Ina Garten],[],[],[],None
7,0001517791,"[[Movies & TV, Movies]]",Praise Aerobics - A low-intensity/high-intesit...,Praise Aerobics [VHS],{'Movies & TV': 557724},[],[],"[(Integrity Music Fitness, Recorder)]",[],None
8,0001527665,"[[Movies & TV, Movies]]",NaN,Peace Child [VHS],{'Movies & TV': 734524},[],[],[],[],None
9,0001516035,"[[Movies & TV, Movies]]",Worship with Don Moen [VHS],Worship with Don Moen [VHS],{'Movies & TV': 435882},[],[],"[(Don Moen, Recorder)]",[],None


In [ ]:
## Run this cell to manually save results
##

save_progress(amazon_products_df, lastItemLookedUp)